# Zonal Statistics

In [2]:
from exactextract import exact_extract


In [ ]:
exact_extract?

In [10]:
city_stats = exact_extract("ndvi.tif", 
                           "new_haven.parquet", 
                           ["mean"], 
                           include_geom = True,
                           include_cols=["label", "grade", "city", "fill"],
                           output="pandas")

city_stats.head()

,label,grade,city,fill,mean,geometry
0,A1,A,New Haven,#76a865,0.709624,"MULTIPOLYGON (((-72.90000 41.36885, -72.90325 ..."
1,A2,A,New Haven,#76a865,0.761685,"MULTIPOLYGON (((-72.89401 41.36331, -72.89256 ..."
2,A3,A,New Haven,#76a865,0.665130,"MULTIPOLYGON (((-72.90981 41.35949, -72.90284 ..."
3,A4,A,New Haven,#76a865,0.751388,"MULTIPOLYGON (((-72.90284 41.35795, -72.89990 ..."
4,A5,A,New Haven,#76a865,0.742981,"MULTIPOLYGON (((-72.89801 41.34930, -72.89883 ..."


In [ ]:
city_stats.to_parquet("new_haven_stats.parquet")

We can bring our data back into ibis,

In [8]:
import ibis
from ibis import _
con = ibis.duckdb.connect(extensions=["spatial"])

city = con.read_parquet("new_haven_stats.parquet")

,grade,ndvi
0,C,0.547228
1,A,0.730811
2,D,0.486641
3,B,0.609324


 and we are finally ready to summarize the data to determine if the spectre of redlining remains: on average, are A grades still greener than D grades?  Construct the rest of the `ibis` code to compute the average NDVI by grade.